## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Hobart,AU,-42.8794,147.3294,49.05,84,75,3.44,broken clouds
1,1,Arraial Do Cabo,BR,-22.9661,-42.0278,75.16,83,40,3.44,scattered clouds
2,2,Tiksi,RU,71.6872,128.8694,30.88,83,100,4.59,overcast clouds
3,3,Ribeira Grande,PT,38.5167,-28.7000,73.80,88,75,6.91,mist
4,4,Hermanus,ZA,-34.4187,19.2345,59.90,77,90,23.02,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Arraial Do Cabo,BR,-22.9661,-42.0278,75.16,83,40,3.44,scattered clouds
3,3,Ribeira Grande,PT,38.5167,-28.7000,73.80,88,75,6.91,mist
5,5,Lagoa,PT,39.0500,-27.9833,71.49,87,0,7.14,clear sky
6,6,Ingham,US,42.6000,-84.3833,75.31,54,1,3.44,clear sky
8,8,Pafos,CY,34.7667,32.4167,75.11,78,20,6.91,few clouds
9,9,Zurrieq,MT,35.8311,14.4742,75.96,83,20,9.22,few clouds
11,11,Hervey Bay,AU,-25.2986,152.8535,68.11,82,100,13.53,moderate rain
13,13,Kaseda,JP,31.4167,130.3167,74.86,83,0,10.27,clear sky
18,18,Atuona,PF,-9.8000,-139.0333,77.86,75,55,19.37,light rain
19,19,Butaritari,KI,3.0707,172.7902,81.54,79,97,16.67,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                311
City                   311
Country                309
Lat                    311
Lng                    311
Max Temp               311
Humidity               311
Cloudiness             311
Wind Speed             311
Current Description    311
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                309
City                   309
Country                309
Lat                    309
Lng                    309
Max Temp               309
Humidity               309
Cloudiness             309
Wind Speed             309
Current Description    309
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Arraial Do Cabo,BR,75.16,scattered clouds,-22.9661,-42.0278,
3,Ribeira Grande,PT,73.80,mist,38.5167,-28.7000,
5,Lagoa,PT,71.49,clear sky,39.0500,-27.9833,
6,Ingham,US,75.31,clear sky,42.6000,-84.3833,
8,Pafos,CY,75.11,few clouds,34.7667,32.4167,
9,Zurrieq,MT,75.96,few clouds,35.8311,14.4742,
11,Hervey Bay,AU,68.11,moderate rain,-25.2986,152.8535,
13,Kaseda,JP,74.86,clear sky,31.4167,130.3167,
18,Atuona,PF,77.86,light rain,-9.8000,-139.0333,
19,Butaritari,KI,81.54,light rain,3.0707,172.7902,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Arraial Do Cabo,BR,75.16,scattered clouds,-22.9661,-42.0278,Pousada Porto Praia
3,Ribeira Grande,PT,73.80,mist,38.5167,-28.7000,Quinta da Meia Eira
5,Lagoa,PT,71.49,clear sky,39.0500,-27.9833,Casa Das Faias
6,Ingham,US,75.31,clear sky,42.6000,-84.3833,Sunset Mobile Home Park
8,Pafos,CY,75.11,few clouds,34.7667,32.4167,Elysium Hotel
9,Zurrieq,MT,75.96,few clouds,35.8311,14.4742,Ta'Bertu Guest House
11,Hervey Bay,AU,68.11,moderate rain,-25.2986,152.8535,Shelly Bay Resort
13,Kaseda,JP,74.86,clear sky,31.4167,130.3167,Sun Stay Kaseda
18,Atuona,PF,77.86,light rain,-9.8000,-139.0333,Villa Enata
19,Butaritari,KI,81.54,light rain,3.0707,172.7902,Isles Sunset Lodge


In [19]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   285
Country                285
Max Temp               285
Current Description    285
Lat                    285
Lng                    285
Hotel Name             285
dtype: int64

In [21]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [23]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))